In [1]:
from sklearn.model_selection import train_test_split
import unittest
import numpy as np
from keras_performer import performerErrorTypeTest_V2 as tfr
import nltk

from os import listdir
from os.path import isfile, isdir, join
from nltk.tokenize import wordpunct_tokenize

nltk.download('punkt')
from nltk.tokenize import RegexpTokenizer
from tensorflow.keras.optimizers import Adam
from keras.utils import to_categorical
import tensorflow as tf
import glob

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\W.R_Chen\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
import csv
def readCSV(file_name):
    errlist=[]
    LBlist=[]
    with open(file_name, newline='') as csvfile:
    #讀取 CSV 檔內容，將每一列轉成一個 dictionary

        rows = csv.DictReader(csvfile)
        for row in rows: 
            RL=list(row.values())
            #print("RL[0]: ", type(RL[0]), "RL[1]: ", type(RL[1]))
            RL[1:] = list(map(int, RL[1:]))
            errs=RL[1:37]
            LB=RL[37:]
            errlist.append(errs)
            LBlist.append(LB)
    return errlist,LBlist


In [3]:
def loadTestTrainData(filename): # e.g., 'test.npy'
    with open(filename, "rb") as f:
        a = np.load(f)
        return a

In [4]:
def find_first_sublist(seq, sublist, start=0):
    length = len(sublist)
    for index in range(start, len(seq)):
        if seq[index:index+length] == sublist:
            return index, index+length

In [5]:
def replace_sublist(seq, sublist, replacement):
    length = len(replacement)
    index = 0
    for start, end in iter(lambda: find_first_sublist(seq, sublist, index), None):
        seq[start:end] = replacement
        index = start + length

In [6]:
def replaceTAGS(x):
    replace_sublist(x, ['<', 'NUM_INT', '>'], ["<NUM_INT>"])
    replace_sublist(x, ['<', 'NUM_FLOAT', '>'], ["<NUM_FLOAT>"])
    replace_sublist(x, ['<', 'STRING', '>'], ["<STRING>"])
    replace_sublist(x, ['<', 'BOC', '>'], ["<BOC>"])
    replace_sublist(x, ['<', 'EOC', '>'], ["<EOC>"])
    replace_sublist(x, ['<', 'BOTM', '>'], ["<BOTM>"])
    replace_sublist(x, ['<', 'BOT', '>'], ["<BOT>"])
    replace_sublist(x, ['<', 'EOT', '>'], ["<EOT>"])
    replace_sublist(x, ['<', 'BOM', '>'], ["<BOM>"])
    replace_sublist(x, ['<', 'EOM', '>'], ["<EOM>"])
    replace_sublist(x, ['<', 'EOTM', '>'], ["<EOTM>"])
    replace_sublist(x, ['<', 'CR', '>'], ["<CR>"])
    return x

In [7]:
def parseSentence(x):	
    tokenizer = RegexpTokenizer(r"[\w']+|[].,:!?;=+-\\*/@#$%^&_(){}~|\"[]")
    tokens=[]
    state="START"
    chrs=""
    for i in range(len(x)):
        #print(ord(x[i]))
        if (ord(x[i])>255):
            inp="U"
        else:
            inp="E"

        if state=="START":
            if inp=="E":
                state="ASCII"
                chrs=x[i]
            else:
                state="UNICODE"
                tokens.append(x[i])

        elif state=="ASCII":
            if inp=="E":
                chrs += x[i]
            else:#U
                state="UNICODE"
                tokens += tokenizer.tokenize(chrs) #wordpunct_tokenize(chrs)  #nltk.word_tokenize(chrs)
                chrs=""
                tokens.append(x[i])

        elif state=="UNICODE":
            if inp=="E":
                state="ASCII"
                chrs=x[i]
            else:
                state="UNICODE"
                tokens.append(x[i])
    if len(chrs)>0:
        tokens += tokenizer.tokenize(chrs) #wordpunct_tokenize(chrs)  # nltk.word_tokenize(chrs) 
    return tokens

In [8]:
def readcode(fname):
    with open(fname,encoding = 'utf-8') as f:
        data = f.read()
        return data

In [9]:
import os
def listdir_fullpath(d):
    return [f for f in os.listdir(d)]

In [10]:
#save model for training
class TestTranslate(unittest.TestCase):
        
    def __init__(self):
        self.source_token_dict = {
            '<PAD>': 0,
            '<START>': 1,
            '<END>': 2,
            '<BOC>': 3,
            '<EOC>': 4,
            '<CR>': 5,
            '<NUM_INT>': 6,
            '<NUM_FLOAT>': 7,
            '<STRING>': 8,
        }
        
    @staticmethod
    def _build_token_dict(token_dict, token_list):
        for tokens in token_list:
            for token in tokens:
                if token not in token_dict:
                    token_dict[token] = len(token_dict)
        return token_dict
    
    def test_translate(self):
        #print("i am here: " )
        #source_file=[]
        #Set Para
        max_javaline_length = 160 #Max number of lines
        #set path
        Output_Path = "Trianing\InputCSV\Split-500-reduce-binary-accuracy"
        Input_Path = "Trianing\InputTxt\Split-500-reduce-binary-accuracy"
        model_for_training_org_path = "Model-for-training-org\ErrorType-Test\Split-500-reduce-binary-accuracy\Max-len-1000"
        model_for_training_path = "Model-for-training\ErrorType-Test\Split-500-reduce-binary-accuracy\Max-len-1000"
        Trained_model_Path = "Trained_models\ErrorType-Test\Split-500-reduce-binary-accuracy\Max-len-1000\Learning-rate-3x0\weight-1vs1-V2"
        source_max_len_name = "source_max_len"
        type_weight = 1
        line_weight = 1
        learning_rate_value = 0.0001
        #get all txt file in input path
        target_max_len = 0
        token_num = 0

        #start training
        import DataGeneratorTrainErrorTypeEdition as DGTrain
        import DataGeneratorValidationErrorTypeEdition as DGValidation
        import DataBuffer as db
        from random import randrange
        source_token_dict_name = "source_token_dict.pickle"
        #load source_token_dict
        source_token_dict = loadDictionary(model_for_training_path + "/" + source_token_dict_name)
        #load source_max_len
        source_max_len = loadDictionary(Trained_model_Path + "/" + source_max_len_name)
        #load training models and their len
        x_train = loadTestTrainData(model_for_training_path + "/" + "x_train_0.npy")
        x_validation = loadTestTrainData(model_for_training_path + "/" + "x_validation_0.npy")
        
        buffer_train_num = len(x_train)
        buffer_val_train_num = len(x_validation)
        #Set model para    
        model = tfr.get_model(max_input_len=(source_max_len),
                              errNum=36,
                              token_num=len(source_token_dict),
                              embed_dim=32, #32, try 32 or 64
                              encoder_num=6, #2 max = 6
                              head_num=4,#4
                              hidden_dim=128, #128
                              dropout_rate=0.05 #0.05
                             )
        #Set losses
        losses = {"error_feed_forward_output1": "binary_crossentropy"}
        #error type weight
        lossWeights = {"error_feed_forward_output1": type_weight}
        metrics = {"error_feed_forward_output1": "binary_accuracy"}
        #metrics = {"error_feed_forward_output1": tf.keras.metrics.Accuracy()}
        
        #set complie para
        model.compile(optimizer=Adam(learning_rate=learning_rate_value), loss=losses, loss_weights=lossWeights, metrics=metrics)
        
        
        #for output
        #for x
        #data_number and block_size = data num
        input_buffer_params = { 
            "data_path": model_for_training_path,
            "data_number": buffer_train_num,
            "data_type": int,
            "block_size": buffer_train_num 
            }
        
        #for input
        #for y
        #data_number and block_size = data num
        """
        output_buffer_params = {
            "data_path": [model_for_training_path, model_for_training_path],
            "data_number": [buffer_train_num, buffer_train_num],
            "data_type": [int, int],
            "block_size": [buffer_train_num, buffer_train_num] 
            }
        """
        output_buffer_params = {
            "data_path": [model_for_training_path],
            "data_number": [buffer_train_num],
            "data_type": [int],
            "block_size": [buffer_train_num] 
            }
        
        #===========================================
        #for output
        #for x
        #data_number and block_size = validation data num
        
        validation_input_buffer_params = { 
            "data_path": model_for_training_path,
            "data_number": buffer_val_train_num,
            "data_type": int,
            "block_size": buffer_val_train_num 
            }
        
        #for input
        #for y
        #data_number and block_size = validation data num
        """
        validation_output_buffer_params = {
            "data_path": [model_for_training_path, model_for_training_path],
            "data_number": [buffer_val_train_num, buffer_val_train_num],
            "data_type": [int, int],
            "block_size": [buffer_val_train_num, buffer_val_train_num] 
            }
        """
        
        validation_output_buffer_params = {
            "data_path": [model_for_training_path],
            "data_number": [buffer_val_train_num],
            "data_type": [int],
            "block_size": [buffer_val_train_num] 
            }
        
        
        #Create Generators
        print("Creating training generator...")
        #give training data num
        """training_generator = DGTrain.DataGeneratorTrain(input_buffer_params,
                                                           output_buffer_params,
                                                           [list(range(buffer_val_train_num)), list(range(buffer_val_train_num))] 
                                                )"""
        training_generator = DGTrain.DataGeneratorTrainErrorTypeEdition(input_buffer_params,
                                                                        output_buffer_params,
                                                                        [list(range(buffer_val_train_num))] 
                                                )
        #Create Generators
        print("Creating validation generator...")
        #give valitdation data num
        """validation_generator = DGValidation.DataGeneratorValidation(validation_input_buffer_params,
                                                                       validation_output_buffer_params,
                                                                      [list(range(buffer_val_train_num)), list(range(buffer_val_train_num))] 
                                                )"""
        validation_generator = DGValidation.DataGeneratorValidationErrorTypeEdition(validation_input_buffer_params,
                                                  validation_output_buffer_params,
                                                  [list(range(buffer_val_train_num))] 
                                                )
        
        
        #''' <-----traing switch
        #Start training
        print("Strat training...")
        
        model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath = Trained_model_Path + "/" + "checkpoint_model.h5",
                                                                       monitor = "val_loss",
                                                                       mode = "min",
                                                                       save_best_only = True
                                                                      )
        
        history = model.fit_generator(generator = training_generator,
                                      epochs = 1000, #100 200 500 3000
                                      verbose = 2, #set visibility
                                      validation_data = validation_generator,
                                      callbacks = [model_checkpoint_callback],
                                     )
        
        
        print("Model training completed...")
        #save history
        print("Saving history...")
        saveDictionary(history.history, Trained_model_Path + "/" + "model_history")
        print("History saving completed...")
        
        #save model
        print("Saving model...")
        model.save(Trained_model_Path + "/" + "test_model1.h5")
        print("Model saving completed...")
        
        #print("history.history.keys: ", history.history.keys())\
        #'''
                
    def getsource_max_lan(self):
        return self.sl
    

In [11]:
def saveDictionary(dt, file):
        import pickle
        a_file = open(file, "wb")
        pickle.dump(dt, a_file)
        a_file.close()

In [12]:
def loadDictionary(file):
        import pickle
        a_file = open(file, "rb")
        dt = pickle.load(a_file)
        return dt

In [13]:
def saveTestTrainData(filename, data): # e.g., 'test.npy'
    with open(filename, 'wb') as f:
        np.save(f, data)

In [14]:
def loadTestTrainData(filename): # e.g., 'test.npy'    
    with open(filename, 'rb') as f:
        a = np.load(f)
        return a

In [15]:
x=TestTranslate()
x.test_translate()

Start Warpping...
Start Warpping...
Start Warpping...
Start Warpping...
Start Warpping...
Start Warpping...
Start Warpping...
Start Warpping...
Start Warpping...
Start Warpping...
Start Warpping...
Start Warpping...
Creating training generator...
Creating validation generator...
Strat training...


C:\Users\W.R_Chen\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/1000
21/21 - 8s - loss: 0.2906 - binary_accuracy: 0.9129 - val_loss: 0.2137 - val_binary_accuracy: 0.9551
Epoch 2/1000


C:\Users\W.R_Chen\anaconda3\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


21/21 - 3s - loss: 0.2057 - binary_accuracy: 0.9551 - val_loss: 0.1998 - val_binary_accuracy: 0.9551
Epoch 3/1000
21/21 - 3s - loss: 0.1954 - binary_accuracy: 0.9552 - val_loss: 0.1914 - val_binary_accuracy: 0.9552
Epoch 4/1000
21/21 - 3s - loss: 0.1906 - binary_accuracy: 0.9552 - val_loss: 0.1879 - val_binary_accuracy: 0.9552
Epoch 5/1000
21/21 - 3s - loss: 0.1874 - binary_accuracy: 0.9551 - val_loss: 0.1853 - val_binary_accuracy: 0.9551
Epoch 6/1000
21/21 - 3s - loss: 0.1850 - binary_accuracy: 0.9552 - val_loss: 0.1844 - val_binary_accuracy: 0.9551
Epoch 7/1000
21/21 - 3s - loss: 0.1828 - binary_accuracy: 0.9552 - val_loss: 0.1814 - val_binary_accuracy: 0.9552
Epoch 8/1000
21/21 - 3s - loss: 0.1825 - binary_accuracy: 0.9552 - val_loss: 0.1810 - val_binary_accuracy: 0.9552
Epoch 9/1000
21/21 - 3s - loss: 0.1827 - binary_accuracy: 0.9551 - val_loss: 0.1840 - val_binary_accuracy: 0.9551
Epoch 10/1000
21/21 - 3s - loss: 0.1830 - binary_accuracy: 0.9551 - val_loss: 0.1810 - val_binary_acc

...

Epoch 707/1000
21/21 - 3s - loss: 3.1594e-04 - binary_accuracy: 0.9999 - val_loss: 1.2470 - val_binary_accuracy: 0.9310
Epoch 708/1000
21/21 - 3s - loss: 3.0526e-04 - binary_accuracy: 0.9999 - val_loss: 1.2467 - val_binary_accuracy: 0.9307
Epoch 709/1000
21/21 - 2s - loss: 3.9787e-04 - binary_accuracy: 0.9999 - val_loss: 1.2574 - val_binary_accuracy: 0.9303
Epoch 710/1000
21/21 - 3s - loss: 3.2980e-04 - binary_accuracy: 1.0000 - val_loss: 1.2454 - val_binary_accuracy: 0.9312
Epoch 711/1000
21/21 - 3s - loss: 3.9816e-04 - binary_accuracy: 0.9999 - val_loss: 1.2585 - val_binary_accuracy: 0.9309
Epoch 712/1000
21/21 - 3s - loss: 4.7210e-04 - binary_accuracy: 1.0000 - val_loss: 1.2185 - val_binary_accuracy: 0.9305
Epoch 713/1000
21/21 - 3s - loss: 2.9915e-04 - binary_accuracy: 1.0000 - val_loss: 1.2575 - val_binary_accuracy: 0.9310
Epoch 714/1000
21/21 - 3s - loss: 2.8429e-04 - binary_accuracy: 0.9999 - val_loss: 1.2403 - val_binary_accuracy: 0.9311
Epoch 715/1000
21/21 - 3s - loss: 2.9975

Epoch 776/1000
21/21 - 3s - loss: 3.8208e-04 - binary_accuracy: 1.0000 - val_loss: 1.3050 - val_binary_accuracy: 0.9315
Epoch 777/1000
21/21 - 3s - loss: 2.7375e-04 - binary_accuracy: 0.9999 - val_loss: 1.2958 - val_binary_accuracy: 0.9302
Epoch 778/1000
21/21 - 3s - loss: 3.3206e-04 - binary_accuracy: 0.9999 - val_loss: 1.2875 - val_binary_accuracy: 0.9313
Epoch 779/1000
21/21 - 3s - loss: 3.3583e-04 - binary_accuracy: 0.9999 - val_loss: 1.3019 - val_binary_accuracy: 0.9309
Epoch 780/1000
21/21 - 3s - loss: 8.2394e-05 - binary_accuracy: 1.0000 - val_loss: 1.3074 - val_binary_accuracy: 0.9310
Epoch 781/1000
21/21 - 3s - loss: 2.6150e-04 - binary_accuracy: 1.0000 - val_loss: 1.2963 - val_binary_accuracy: 0.9308
Epoch 782/1000
21/21 - 3s - loss: 2.2678e-04 - binary_accuracy: 0.9999 - val_loss: 1.2979 - val_binary_accuracy: 0.9312
Epoch 783/1000
21/21 - 3s - loss: 2.8392e-04 - binary_accuracy: 1.0000 - val_loss: 1.3157 - val_binary_accuracy: 0.9309
Epoch 784/1000
21/21 - 2s - loss: 3.1875

Epoch 845/1000
21/21 - 3s - loss: 2.8206e-04 - binary_accuracy: 0.9999 - val_loss: 1.3490 - val_binary_accuracy: 0.9305
Epoch 846/1000
21/21 - 3s - loss: 3.2240e-04 - binary_accuracy: 0.9999 - val_loss: 1.3780 - val_binary_accuracy: 0.9314
Epoch 847/1000
21/21 - 3s - loss: 1.9948e-04 - binary_accuracy: 1.0000 - val_loss: 1.3566 - val_binary_accuracy: 0.9309
Epoch 848/1000
21/21 - 3s - loss: 1.7677e-04 - binary_accuracy: 1.0000 - val_loss: 1.3573 - val_binary_accuracy: 0.9310
Epoch 849/1000
21/21 - 3s - loss: 2.5322e-04 - binary_accuracy: 1.0000 - val_loss: 1.3549 - val_binary_accuracy: 0.9306
Epoch 850/1000
21/21 - 3s - loss: 1.8759e-04 - binary_accuracy: 1.0000 - val_loss: 1.3529 - val_binary_accuracy: 0.9308
Epoch 851/1000
21/21 - 3s - loss: 1.9865e-04 - binary_accuracy: 0.9999 - val_loss: 1.3414 - val_binary_accuracy: 0.9309
Epoch 852/1000
21/21 - 3s - loss: 2.4540e-04 - binary_accuracy: 1.0000 - val_loss: 1.3491 - val_binary_accuracy: 0.9310
Epoch 853/1000
21/21 - 3s - loss: 1.8717

Epoch 914/1000
21/21 - 3s - loss: 2.1376e-04 - binary_accuracy: 1.0000 - val_loss: 1.4213 - val_binary_accuracy: 0.9297
Epoch 915/1000
21/21 - 3s - loss: 1.7809e-04 - binary_accuracy: 0.9999 - val_loss: 1.4162 - val_binary_accuracy: 0.9298
Epoch 916/1000
21/21 - 3s - loss: 1.7503e-04 - binary_accuracy: 1.0000 - val_loss: 1.4219 - val_binary_accuracy: 0.9305
Epoch 917/1000
21/21 - 3s - loss: 1.8154e-04 - binary_accuracy: 1.0000 - val_loss: 1.4202 - val_binary_accuracy: 0.9299
Epoch 918/1000
21/21 - 3s - loss: 1.9143e-04 - binary_accuracy: 1.0000 - val_loss: 1.4234 - val_binary_accuracy: 0.9303
Epoch 919/1000
21/21 - 3s - loss: 1.8612e-04 - binary_accuracy: 1.0000 - val_loss: 1.4323 - val_binary_accuracy: 0.9302
Epoch 920/1000
21/21 - 2s - loss: 1.5404e-04 - binary_accuracy: 0.9999 - val_loss: 1.4299 - val_binary_accuracy: 0.9298
Epoch 921/1000
21/21 - 3s - loss: 2.4183e-04 - binary_accuracy: 0.9999 - val_loss: 1.4239 - val_binary_accuracy: 0.9301
Epoch 922/1000
21/21 - 3s - loss: 1.9437

Epoch 983/1000
21/21 - 2s - loss: 2.5330e-04 - binary_accuracy: 1.0000 - val_loss: 1.4497 - val_binary_accuracy: 0.9305
Epoch 984/1000
21/21 - 2s - loss: 1.8312e-04 - binary_accuracy: 1.0000 - val_loss: 1.4534 - val_binary_accuracy: 0.9305
Epoch 985/1000
21/21 - 2s - loss: 2.7773e-04 - binary_accuracy: 0.9999 - val_loss: 1.4657 - val_binary_accuracy: 0.9301
Epoch 986/1000
21/21 - 2s - loss: 1.5029e-04 - binary_accuracy: 1.0000 - val_loss: 1.4460 - val_binary_accuracy: 0.9304
Epoch 987/1000
21/21 - 2s - loss: 2.1315e-04 - binary_accuracy: 0.9999 - val_loss: 1.4761 - val_binary_accuracy: 0.9304
Epoch 988/1000
21/21 - 2s - loss: 2.7873e-04 - binary_accuracy: 1.0000 - val_loss: 1.4707 - val_binary_accuracy: 0.9305
Epoch 989/1000
21/21 - 3s - loss: 1.6539e-04 - binary_accuracy: 0.9999 - val_loss: 1.4779 - val_binary_accuracy: 0.9305
Epoch 990/1000
21/21 - 3s - loss: 2.0821e-04 - binary_accuracy: 1.0000 - val_loss: 1.4704 - val_binary_accuracy: 0.9305
Epoch 991/1000
21/21 - 2s - loss: 1.5316